# Local Data Tutorial
In this tutorial, rather than running real models and configurations over MIMIC-IV, we'll work with a set of
local, synthetic files distributed with this repository, with the goal being to fully explore the details of
this pipeline. This tutorial will consist of both content on this page, running certain scripts on one's local
machine, and some jupyter notebooks. We will walk through the entire pipeline with these local examples and
discuss limitations of the pipeline, details of classes, scripts, etc.

## Synthetic Data
For this tutorial, we'll use the three synthetic data files distributed in the [sample_data/raw](<>) folder in
the repository:

In [1]:
!ls --color -lah raw

total 55M
drwxrwxr-x 2 mmd mmd 4.0K Jul 14 12:42 .
drwxrwxr-x 5 mmd mmd 4.0K Jul 17 11:42 ..
-rw-rw-r-- 1 mmd mmd 3.6M Jul 17 11:40 admit_vitals.csv
-rw-rw-r-- 1 mmd mmd  52M Jul 17 11:40 labs.csv
-rw-rw-r-- 1 mmd mmd 4.2K Jul 17 11:40 subjects.csv


```{note}
To see how those files are generated, look at [sample_data/generate_synthetic_data.ipynb]().
```

These files contain the following data:

### `subjects.csv`

This file contains per-subject data. It has one row per subject, with each row containing a subject identifier (here called "`MRN`"), a date of birth ("`dob`"), the subject's eye color (`eye_color`), and the subject's height ("`height`"):

```{note}
To examine these csv files locally, we use the `csvkit` unix command line package. You can learn more about it [here](https://csvkit.readthedocs.io/en/latest/index.html).
```

In [2]:
!csvlook -d "," --max-rows=5 raw/subjects.csv

|     MRN |        dob | eye_color |   height |
| ------- | ---------- | --------- | -------- |
| 310,243 | 1981-07-28 | GREEN     | 178.768… |
| 384,198 | 1985-04-15 | BROWN     | 168.319… |
| 520,533 | 1979-04-15 | BROWN     | 165.836… |
| 850,710 | 1970-08-08 | HAZEL     | 159.722… |
| 379,433 | 1971-06-25 | BLUE      | 153.749… |
|     ... |        ... | ...       |      ... |


### `admit_vitals.csv`

This file contains dynamic data quantifying both fictional subject hospital admissions, and fictional vitals signs measured for those subjects. Each row of this file records a unique vitals sign measurement for a patient, affiliated with the associated admission listed in the row. This means that admission level information is _heavily duplicated_ within this file, which is a phenomena sometimes observed in real data, and something we'll need to account for in our pipeline's setup.

In [3]:
!csvlook -d "," --max-rows=5 raw/admit_vitals.csv

|       MRN | admit_date           | disch_date           | department | vitals_date          |    HR |  temp |
| --------- | -------------------- | -------------------- | ---------- | -------------------- | ----- | ----- |
| 1,549,363 | 01/04/2010, 06:36:31 | 01/14/2010, 11:41:31 | ORTHOPEDIC | 01/11/2010, 14:18:35 |  77.1 |  96.3 |
|   415,881 | 02/11/2010, 04:59:07 | 02/14/2010, 07:56:23 | ORTHOPEDIC | 02/11/2010, 10:34:17 | 148.5 |  95.6 |
|    42,335 | 03/06/2010, 05:33:18 | 03/16/2010, 05:09:11 | CARDIAC    | 03/13/2010, 10:47:39 |  46.7 | 101.0 |
| 1,516,810 | 02/11/2010, 23:23:26 | 02/22/2010, 23:59:04 | CARDIAC    | 02/12/2010, 16:58:44 |  94.2 |  95.2 |
|   928,262 | 02/19/2010, 03:51:12 | 02/20/2010, 23:32:31 | CARDIAC    | 02/19/2010, 18:08:30 |  77.3 |  99.7 |
|       ... | ...                  | ...                  | ...        | ...                  |   ... |   ... |


### `labs.csv`

This file contains dynamic data quantifying fictional subject laboratory test measurements. Each row of this file contains a record of a particular lab test measured for a subject. Note that the lab data is not organized into separate columns for each lab; rather each row contains a pair of a lab test name and the associated value; this is what we call in ESGPT a "multivariate regression" column encoding.

In [4]:
!csvlook -d "," --max-rows=5 raw/labs.csv

|     MRN | timestamp           | lab_name   | lab_value |
| ------- | ------------------- | ---------- | --------- |
| 928,262 | 09:31:08-2010-03-04 | SpO2       |     50.00 |
| 689,012 | 04:28:32-2010-08-18 | creatinine |      1.45 |
| 706,423 | 01:18:05-2010-07-22 | SpO2       |     51.00 |
| 383,358 | 16:00:02-2010-05-07 | SpO2       |     55.00 |
| 407,452 | 15:36:06-2010-09-27 | SpO2       |     50.00 |
|     ... | ...                 | ...        |       ... |


# Processing Synthetic Data with ESGPT

Now that we see the form of this synthetic data, we can examine how to process it with Event Stream GPT. From
the base directory of the ESGPT repository, we can run the following command:

```bash
PYTHONPATH=$(pwd):$PYTHONPATH ./scripts/build_dataset.py \
	--config-path="$(pwd)/sample_data/" \
	--config-name=dataset \
	"hydra.searchpath=[$(pwd)/configs]"
```

```{note}
This script, like all built-in ESGPT scripts, uses [Hydra](https://hydra.cc/), a configuration file and experiment run-script library. In hydra, all scripts can take as input a set of composable configuration files which can be overwritten via files or viat he command line. If you aren't already familiar with Hydra, you should read through some of their examples or tutorials to gain some familiarity with their system.
```

Before we actually run this command, we need to do 2 things:
  1. Decide what we _want_ the command to do, conceptually.
  2. Understand what we're _telling_ the library to do, via its input arguments.
  
## What do we _want_ to happen?
We can see that our synthetic data has a few different kinds of things happening to these subjects. In the ESGPT data model, we want to organize this data so that we clearly know who our subjects are, quantify when things happen to those subjects, and record in a sparse manner what is happening to those patients. Let's list a few more specific desiderata:
  1. We should expect our system to quantify those subjects in our synthetic data that meet our inclusion criteria (which we haven't yet specified).
  2. The system should bucket all interactions for subjects into appropriately defined events, across admissions, discharges, vitals signs, and laboratory tests.
  3. The system should learn appropriate categorical vocabularies, numerical outlier detector models, numerical normalization models, for the various measurements we want to extract (which we haven't yet specified).
  4. The system should produce "deep-learning friendly" representations of these data.

```{note}
A quick tangent -- what do we mean by "deep-learning friendly" representations of these data? Well, right now, if we were to try to run these data through _any_ deep-learning system for longitudinal data, we'd need to re-format these data such that it is easy to efficiently (ideally $O(1)$) retrieve all data corresponding to a single subject in an organized timeseries format that we can then efficiently (meaning in a manner requiring minimal GPU memory) pass into a sequential neural network. 

In the current representation, this retrieval process would not be $O(1)$; instead, if we didn't modify the data's organization at all, for each new MRN, we'd need to select from each data file all those rows with that MRN (each selection being an $O(N)$ operation), and then we would need to subsequently sort all the temporal data by timestamp (another $O(L\ln(L))$ operation).

Similarly, if we use a naive, dense encoding of the data per measurement for our DL representation, this will be very wasteful in terms of GPU memory, as each record will need to occupy memory proportionate to the total number of possible measurements we could observe in our data (e.g., the total number of lab tests, plus the total number of vitals signs, plus the total number of admission departments, etc.). Instead, a sparse encoding should be used.

These two properties are exactly what we mean by a "deep-learning friendly" representation of the data.
```

We can see that there are several questions posed by these desiderata that we need to answer, such as:
  1. What are our inclusion criteria?
  2. How should we bucket interactions into events?
  3. What measurements do we want to extract?
  4. How do we want to define "outliers"?
  5. How do we define "appropriate categorical vocabularies"?
  6. How do we want to normalize numerical measurements?
  
To start us off, let's use the following answers:
  1. We'll include all subjects who have at least 3 events, with no other inclusion/exclusion criteria.
  2. We'll define an "event" to be any interactions happening to a patient within a 1 hour period. We'll bucket these interactions together starting at the earliest event.
  3. Ideally, we'd like to extract _all_ measurements. As we'll see, however, due to a limitation in the current version of ESGPT, we'll extract all measurements except for the patient's height. In particular, we'll extract the occurrence of admissions, discharges, vitals signs, and laboratory tests, as well as the subject's age, eye color,  admission department, the values recorded for HR and temperature, and all lab test values.
  4. We'll use a very simple outlier model, that excludes numerical data as outliers if their values exceed 1.5 standard deviations from the mean. This is an extremely aggressive cutoff only suitable for this synthetic data setting.
  5. We'll keep any categorical observation as a vocabulary element if it occurs at least 5 times.
  6. We'll normalize our numerical observations to have zero mean and unit variance.
  
## Telling the pipeline what to do: input config
Now that we have some basic idea of what we want the pipeline to do, let's examine the input configuration file that we pass to the dataset script:

In [5]:
!cat dataset.yaml

defaults:
  - dataset_base
  - _self_

# So that it can be run multiple times without issue.
do_overwrite: True

cohort_name: "sample"
subject_id_col: "MRN"
raw_data_dir: "./sample_data/raw/"
save_dir: "./sample_data/processed/${cohort_name}"

DL_chunk_size: null

inputs:
  subjects:
    input_df: "${raw_data_dir}/subjects.csv"
  admissions:
    input_df: "${raw_data_dir}/admit_vitals.csv"
    start_ts_col: "admit_date"
    end_ts_col: "disch_date"
    ts_format: "%m/%d/%Y, %H:%M:%S"
    event_type: ["OUTPATIENT_VISIT", "ADMISSION", "DISCHARGE"]
  vitals:
    input_df: "${raw_data_dir}/admit_vitals.csv"
    ts_col: "vitals_date"
    ts_format: "%m/%d/%Y, %H:%M:%S"
  labs:
    input_df: "${raw_data_dir}/labs.csv"
    ts_col: "timestamp"
    ts_format: "%H:%M:%S-%Y-%m-%d"

measurements:
  static:
    single_label_classification:
      subjects: ["eye_color"]
  functional_time_dependent:
    age:
      functor: AgeFunctor
      necessary_static_measurements: { "dob": ["timestamp", "%m/%d/

There are a number of sections in this file. Firstly, the first three lines ensure this config builds on the defaults provided with the ESGPT library, via Hydra's normal mechanisms. If you aren't familiar with this syntax, check out the [Hydra documentation](https://hydra.cc/docs/1.3/advanced/defaults_list/).

Next, there is a section defining some overarching variables and a section defining our input sources. We can see this section details the paths to each of our input files as well as the formatting used for (most of) the timestamps within these files. Note that this section makes use of [Hydra/OmegaConf's Interpolations](https://omegaconf.readthedocs.io/en/2.3_branch/grammar.html#interpolation-strings) to simplify the specification of the file paths used. 

```{warning}
Two parameters in this section are required: `subject_id_col`, and `cohort_name`. This will be explored in more detail later in this tutorial.
```

Next, we have a section defining the various measurements we'll exctract in this dataset. We can see we specify each of the measurements we discussed above:
  1. `eye_color` is extracted as a `static`, `single_label_classification` measure. 
  2. `age` is extracted as a `functional_time_dependent` measure, leveraging the date-of-birth column `dob`. _Note that this is where we define the timestamp format for the `dob` column, as it is a timestamp formatted static column!_
  3. `department` is extracted as a `dynamic`, `multi_label_classification` measure.
  4. `HR`, and `temp` are extracted as `dynamic`, `univariate_regression` measures.
  5. `lab_name` and `lab_value` are extracted as a single `dynamic`, `multivariate_regression` measure.
  
```{note}
The terms `static`, `functional_time_dependent`, & `dynamic` and `single_label_classification`, `multi_label_classification`, `univariate_regression`, and `multivariate_regression`, are defined enumerations in the `EventStream.data.config` sub-module, and dictate where measurements are stored and how they are pre-processed.
```
  
Finally, we have the remaining set of parameters, which define our inclusion-exclusion criteria (by specifying `min_events_per_subject`), our outlier and normalizer model configuration parameters (`normalization` being omitted here as what we want is the default value), our filtering thresholds for vocabulary elements, and the aggregation time-scale for events.

### What else _could_ we have specified?
To better understand the structure of this input specification, let's explore this input configuration file in a bit more detail. To start with, let's look at what the default, base config contains (the config we inherit from in the defaults list):

In [6]:
!cat ../configs/dataset_base.yaml

defaults:
  - outlier_detector_config: stddev_cutoff
  - normalizer_config: standard_scaler
  - _self_

cohort_name: ???
save_dir: ${oc.env:PROJECT_DIR}/data/${cohort_name}
subject_id_col: ???
seed: 1
split: [0.8, 0.1]
do_overwrite: False
DL_chunk_size: 20000
min_valid_vocab_element_observations: 25
min_valid_column_observations: 50
min_true_float_frequency: 0.1
min_unique_numerical_observations: 25
min_events_per_subject: 20
agg_by_time_scale: null

hydra:
  job:
    name: build_${cohort_name}
  run:
    dir: ${save_dir}/.logs
  sweep:
    dir: ${save_dir}/.logs


We can see there are some parameters we're familiar with and some we're not. Firstly, we can see that this default base config marks `cohort_name` and `subject_id_col` with `???`. This is the OmegaConf provided value to represent a value that _needs to be overwritten_ in downstream usage. This is why those two parameters are mandatory. This config also has variables for the seed, split size, and some hydra-internal parameters. Further, it points to two  further default configs for the outlier detector and normalizer:

In [7]:
!cat ../configs/outlier_detector_config/stddev_cutoff.yaml

cls: stddev_cutoff
stddev_cutoff: 5.0


In [8]:
!cat ../configs/normalizer_config/standard_scaler.yaml

cls: standard_scaler


These are both quite simple, but show how the final config will be constructed from these values. 

One thing that is notably missing from this broader structure is any notion of included `inputs` or `measurements` sections. To understand how we can further specify our config, we need to understand how we could modify those sections as well.

#### Inputs
This section allows us to specify which input data frames should be read, and from where. The `inputs:` option should be an object whose keys are the names of input sources and whose values are configuration for those inputs. Currently, two input formats are possible:
  1. The `input_df` format, which is used in this synthetic example. This format has an input configuration that contains the `input_df:` key whose value is a file path pointing to a `csv` or `parquet` data-frame file on disk that contains that input source's data. For example:
  ```yaml
admissions:
    input_df: "${raw_data_dir}/admit_vitals.csv"
    start_ts_col: "admit_date"
    end_ts_col: "disch_date"
    ts_format: "%m/%d/%Y, %H:%M:%S"
    event_type: ["OUTPATIENT_VISIT", "ADMISSION", "DISCHARGE"]
  ```
  2. The `query` format, which is used in the MIMIC-IV tutorial. In this format, you must specify a `query` parameter. This parameter can either be a string query or a list of string queries, in which case you must specify a global `connection_uri` parameter detailing the URI of the database to which you wish to query (In the [connector-x](https://github.com/sfu-db/connector-x) format), or a dictionary, with keys and values specifying parameters of the [`EventStream.data.dataset_polars.Query`]() object. For example:
  ```yaml
patients:
    query: |-
      SELECT subject_id, gender, to_date((anchor_year-anchor_age)::CHAR(4), 'YYYY') AS year_of_birth
      FROM mimiciv_hosp.patients
      WHERE subject_id IN (
        SELECT long_icu.subject_id FROM (
          (
            SELECT subject_id FROM mimiciv_icu.icustays WHERE los > ${min_los}
          ) AS long_icu INNER JOIN (
            SELECT subject_id
            FROM mimiciv_hosp.admissions
            GROUP BY subject_id
            HAVING COUNT(*) > ${min_admissions}
          ) AS many_admissions
          ON long_icu.subject_id = many_admissions.subject_id
        )
      )
    must_have: ["gender", "year_of_birth"]
  ```
  
Each input can also have a number of other keys and values, including:

##### Timestamp & Event-type Specification.
For non-static data sources, the keys `ts_col` or `start_ts_col` and `end_ts_col` specify the name of the column (or columns) containing the timestamp for the event, and `ts_format` the format of that timestamp. `ts_col` is used for data-sources where each row represents one event, and `start_`/`end_ts_col` for data-sources where each row specifies a range in time. For example, in our synthetic config,
```yaml
admissions:
    input_df: "${raw_data_dir}/admit_vitals.csv"
    start_ts_col: "admit_date"
    end_ts_col: "disch_date"
    ts_format: "%m/%d/%Y, %H:%M:%S"
    event_type: ["OUTPATIENT_VISIT", "ADMISSION", "DISCHARGE"]
```

specifies a range event, where the start timestamp is stored in `admit_date` and the end timestamp in `disch_date`, formatted as `"%m/%d/%Y, %H:%M:%S"`. In contrast,

```yaml
labs:
    input_df: "${raw_data_dir}/labs.csv"
    ts_col: "timestamp"
    ts_format: "%H:%M:%S-%Y-%m-%d"
```

captures data where each row is a single-timepoint event, with timestamp stored in `"%H:%M:%S-%Y-%m-%d"` format in column `timestamp`.

You can also explicitly set the type of each event. Events' types in ESGPT are categorical variables defined by the user that are used to dictate any intra-event causal dependency graphs in downstream models, can be used to help define downstream tasks, and are otherwise used to analyze and describe data. When using the pre-defined build dataset script, they can either be explicitly set or are automatically inferred from the name of the input block. For example, in the examples given above, the `labs:` block produces an input source with the event type `LAB` (the singular, upper-cased inflection of the name of the block, `'labs'`), and `admissions` (being a range event) produces events of type `'OUTPATIENT_VISIT'` when `admit_date == disch_date` and `'ADMISSION'` on `admit_date` and `'DISCHARGE'` on `disch_date`.
```{note}
For range events, the default event types are defined to be `*_EQ`, `*_START`, and `*_END`, where `*` is the singular, upper-cased inflection of the input block name.
```

Event types can also be defined to be column dependent. For example, in this config example (which is not part of our current synthetic example config), we see that event types are defined to take on the value of the column `'visit_occurrence_concept_name'` for the case that the start and end times are the same and for start events, but the static `'Drug Stop'` for end events.

```yaml
drugs:
    input_df: "${raw_data_dir}/drug.parquet"
    start_ts_col: "drug_exposure_start_datetime"
    end_ts_col: ["drug_exposure_end_datetime", "verbatim_end_date"]
    event_type: ["COL:visit_occurrence_concept_name", "COL:visit_occurrence_concept_name", "Drug Stop"]
    start_columns: {"standard_concept_name": "drug", "drug_type_concept_name": "drug_type"}
    end_columns: 
        standard_concept_name: drug
        drug_type_concept_name: drug_type
        stop_reason: drug_stop_reason
```

##### Filtering
You can also specify a simple filter used for a given input source. For example, in the `patients` block in the MIMIC-IV example, we specify that valid rows must have `'gender'` and `'year_of_birth'` defined and non-null. This is another way to enforce cohort inclusion/exclusion criteria. The filter object can either be a list of strings, in which case those columns must have non-null values, or a dictionary from column names to either the boolean `True` (indicating the column must be present and non-null) or lists of allowable values for that column.

##### Measurement columns to extract
You can also specify which measurements should be extracted to associate with a given input data source. Largely, this information will be determined automatically based on the `measurements` section of the config; however, it can be specified explicitly as well. The most common case this would be done is to differentiate different measurements to associate with `start` and `end` events for range events or to re-name measurements from their input column names to new names for internal use (this can be done not only for cosmetic reasons, but so as to unify or disentangle measurements across different input files). For example, in the `drugs:` example shown above, the columns `standard_concept_name` and `drug_type_concept_name` are both used for both start and end events, and are renamed to `'drug'`, and `'drug_type'` in both cases, whereas `stop_reason` is used only for end events (and is renamed to `drug_stop_reason`).
#### Measurements Section
The `measurements:` block lists all the actual measurements that should be extracted from those input sources, broken down into categories based on their `temporality` and `modality` (see `EventStream.data.types.TemporalityType` and `EventStream.data.types.DataModality`, respectively). 

The only non-standard portion of this block corresponds to the `functional_time_dependent` block, which specifies measurements whose values are _not_ stored in the raw input data by default, but are instead computable dynamically given per-subject static data and the timestamps of other events that occur in the data. A good example is a subject's age, which is included in our synthetic configuration. Given a subject's date-of-birth and the timestamp of any other event, we can dynamically compute the subject's age as of that event, which is exactly what the `EventStream.data.time_dependent_functor.AgeFunctor` does.

The structure of this config section is
```yaml
functional_time_dependent:
  output_measurement_name:
      functor: ??? # The functor that is used for this measurement. Must be in `EventStream.data.config.MeasurementConfig.FUNCTORS`
      necessary_static_measurements: { "static_measurement_column": ??? } # column name: column formatting info
      kwargs: { kwarg: kwval } # Keyword args to pass to functor constructor.
```

Currently, only [`AgeFunctor`]() and [`TimeOfDayFunctor`] are pre-defined and supported, but this can be extended by the user by directly adding new functors to the [`EventStream.data.config.MeasurementConfig`]() object.

## Running the Command
Now that we understand the setup a bit better, let's run the actual command:

```bash
PYTHONPATH=$(pwd):$PYTHONPATH ./scripts/build_dataset.py \
	--config-path="$(pwd)/sample_data/" \
	--config-name=dataset \
	"hydra.searchpath=[$(pwd)/configs]"
```

You should see as output the printed line `Empty new events dataframe of type OUTPATIENT_VISIT!`, but
otherwise nothing. Before we proceed further, let's break down what this process has done, and how it could do
things differently. 

Firstly, let's take a look at what is produced in the output folder itself.

In [34]:
!du -sh processed/sample/

30M	processed/sample/


In [32]:
!ls --color -lah -R processed/sample

processed/sample:
total 19M
drwxrwxr-x 5 mmd mmd 4.0K Jul 14 17:14 .
drwxrwxr-x 3 mmd mmd 4.0K Jul 14 17:14 ..
-rw-rw-r-- 1 mmd mmd 1.8K Jul 17 11:41 config.json
drwxrwxr-x 2 mmd mmd 4.0K Jul 14 17:14 DL_reps
-rw-rw-r-- 1 mmd mmd  12M Jul 17 11:41 dynamic_measurements_df.parquet
-rw-rw-r-- 1 mmd mmd 5.2K Jul 17 11:41 E.pkl
-rw-rw-r-- 1 mmd mmd 7.1M Jul 17 11:41 events_df.parquet
-rw-rw-r-- 1 mmd mmd 1.5K Jul 17 11:40 hydra_config.yaml
-rw-rw-r-- 1 mmd mmd 2.9K Jul 17 11:41 inferred_measurement_configs.json
drwxrwxr-x 2 mmd mmd 4.0K Jul 14 17:14 inferred_measurement_metadata
-rw-rw-r-- 1 mmd mmd 1.7K Jul 17 11:40 input_schema.json
drwxrwxr-x 3 mmd mmd 4.0K Jul 14 17:14 .logs
-rw-rw-r-- 1 mmd mmd 2.7K Jul 17 11:41 subjects_df.parquet
-rw-rw-r-- 1 mmd mmd  772 Jul 17 11:41 vocabulary_config.json

processed/sample/DL_reps:
total 12M
drwxrwxr-x 2 mmd mmd 4.0K Jul 14 17:14 .
drwxrwxr-x 5 mmd mmd 4.0K Jul 14 17:14 ..
-rw-rw-r-- 1 mmd mmd 1.2M Jul 17 11:41 held_out_0.parquet
-rw-rw-r-- 1 mmd m

Now, let's walk through what happens when we run this script, step-by-step, and how each of the files listed above are produced.

### Step 1: Config Parsing
First, the script parses our input config file into a slightly refined structured form, then passes that as input to the `EventStream.data.dataset_polars.Dataset` constructor.

To see what this process looks like, we can inspect one portion of the output of the overall script, which we can find in the [sample_data/processed/sample]() directory; in particular, the `input_schema.json` file.
```{note}
The `sample_data/processed/sample` directory is the `save_dir` key in our `dataset.yaml` configuration file.
```

In [33]:
!cat processed/sample/input_schema.json | python -m json.tool

{
    "static": {
        "input_df": "./sample_data/raw//subjects.csv",
        "type": "static",
        "event_type": null,
        "subject_id_col": "MRN",
        "ts_col": null,
        "start_ts_col": null,
        "end_ts_col": null,
        "ts_format": null,
        "start_ts_format": null,
        "end_ts_format": null,
        "data_schema": [
            {
                "eye_color": "categorical",
                "dob": [
                    "dob",
                    [
                        "timestamp",
                        "%m/%d/%Y"
                    ]
                ]
            }
        ],
        "start_data_schema": null,
        "end_data_schema": null,
        "must_have": []
    },
    "dynamic": [
        {
            "input_df": "./sample_data/raw//admit_vitals.csv",
            "type": "range",
            "event_type": [
                "OUTPATIENT_VISIT",
                "ADMISSION",
                "DISCHARGE"
            ],
            "subjec

This object, stored in JSON format, is an instance of the `EventStream.data.config.DatasetSchema` object; interested readers can read more about it's specific formatting requirements there. We can see that this contains much of the same information as was in the initial `dataset.yaml` config shown above, now with some additional data added as well, such as recognizing that the `"lab_name"` column should be read in as a categorical type and `"lab_value"` as a float type.

Beyond the input data schema, the model also writes out the ESGPT's input overall config object to disk, which stores information about which measurements the pipeline is instructed to extract. That object is stored in `config.json`:

In [10]:
!cat processed/sample/config.json | python -m json.tool

{
    "measurement_configs": {
        "eye_color": {
            "name": "eye_color",
            "temporality": "static",
            "modality": "single_label_classification",
            "observation_frequency": null,
            "functor": null,
            "vocabulary": null,
            "values_column": null,
            "_measurement_metadata": null
        },
        "department": {
            "name": "department",
            "temporality": "dynamic",
            "modality": "multi_label_classification",
            "observation_frequency": null,
            "functor": null,
            "vocabulary": null,
            "values_column": null,
            "_measurement_metadata": null
        },
        "HR": {
            "name": "HR",
            "temporality": "dynamic",
            "modality": "univariate_regression",
            "observation_frequency": null,
            "functor": null,
            "vocabulary": null,
            "values_column": null,
            "_measu

Again, much of this information is simply a more verbose re-arrangement of the data specified in `dataset.yaml`. Notably, no information about the measurements has yet been filled in from the data, though it will eventually be added.

```{note}
This config structure illustrates a capability of the pipeline outside of the traditional input script format; namely, if one constructs the full config manually, one can pre-specify various measurement specific values (such as vocabulary, normalization parameters, etc.) to be used over what would be inferred from the data.
```

There is also the full, expanded hydra config stored in `hydra_config.yaml`, which can help aid in reproducibility.

```{note}
The final input to the constructor of the `EventStream.data.dataset_polars.Dataset` class can be seen in the documentation for its base class, `EventStream.data.dataset_base.DatasetBase` and takeas input:
  1. A `config` object (like that shown in JSON form above.
  2. Either the `subjects_df`, `events_df`, and `dynamic_measurements_df` dataframes directly or an `input_schema` `EventStream.data.config.DatasetSchema` object which is shown in `input_schema.json` above, which is used to construct the three dataframes from source. Currently, the immediate extraction output is not written to disk at all, so we can't directly inspect the `subjects_df`, `events_df`, and `dynamic_measurements_df` that result from our `input_schema`, but we can see their relative structure from the final, pre-processed dataframes which are written to disk, which we'll explore next.
```

#### Step 2: Data reading and pre-processing
After normalizing the input configs, the pipeline next extracts the data from source and performs pre-processing on these dataframes. This pre-processing encompasses several steps, including:
  1. Minimizing data types to minimizing memory/disk footprint.
  2. Splitting data into train, hyperparameter tuning, and held out test sets.
  3. Identifying categorical variable vocabularies.
  4. Converting appropriate numerical variables to categorical.
  5. Fitting numerical outlier detectors and normalizers.
  6. Normalizing numerical data, removing outliers and infrequent vocabulary elements, and writing out processed `subjects_df`, `events_df`, and `dynamic_measurements_df` parquet files.

##### Pre-processed Data Frames
After this process is complete, we gain the following three files:

```{note}
We'll inspect the files manually here, but you can also load the dataset object and inspect them that way, which we'll do below.
```

In [11]:
!ls --color -lah processed/sample/*_df.parquet

-rw-rw-r-- 1 mmd mmd  12M Jul 17 11:41 processed/sample/dynamic_measurements_df.parquet
-rw-rw-r-- 1 mmd mmd 7.1M Jul 17 11:41 processed/sample/events_df.parquet
-rw-rw-r-- 1 mmd mmd 2.7K Jul 17 11:41 processed/sample/subjects_df.parquet


In [12]:
# We use polars to look at these parquet files:
import polars as pl
pl.Config.set_tbl_cols(7)

display(pl.scan_parquet('processed/sample/subjects_df.parquet').head(4).collect())

subject_id,MRN,eye_color,dob
u8,cat,cat,datetime[μs]
0,"""310243""","""GREEN""",1981-07-28 00:00:00
1,"""384198""","""BROWN""",1985-04-15 00:00:00
2,"""520533""","""BROWN""",1979-04-15 00:00:00
3,"""850710""","""HAZEL""",1970-08-08 00:00:00


The subjects dataframe `subjects_df` contains subject IDs (which have been re-named and normalized to occupy the minimal possible `uint` type (here `uint8`), and contains a categorical `eye_color` column for our static measurement, but `height` has been dropped as it wasn't included in our config.

In [13]:
display(pl.scan_parquet('processed/sample/events_df.parquet').head(4).collect())

event_id,subject_id,timestamp,event_type,age,age_is_inlier
u32,u8,datetime[μs],cat,f64,bool
0,0,2010-06-24 13:23:00,"""ADMISSION&VITA…",-0.558276,true
1,0,2010-06-24 14:23:00,"""VITAL&LAB""",-0.55825,true
2,0,2010-06-24 15:23:00,"""VITAL&LAB""",-0.558224,true
3,0,2010-06-24 16:23:00,"""VITAL&LAB""",-0.558199,true


The events dataframe `events_df` contains event IDs, subject IDs, timestamps, event types, and our only functional time dependent measurement, `age`, in normalized form, alongside an inlier/outlier indicator column. We can also see several other properties:
  1. That these data are sorted, first by `subject_id` then by `event_id` (equivalently, by `timestamp`).
  2. That event timestamps are separated by precisely 1 hour, which was our input aggregation window.
  3. That event types have been aggregated into merged categories during aggregation. E.g., event 1 with event type `VITAL&LAB` reflects that events of type `VITAL` and `LAB` have been merged together. This is to ensure that no subject has two distinct events at the same timestamp.

In [14]:
df = pl.scan_parquet('processed/sample/dynamic_measurements_df.parquet')
print("Dynamic Measurement Columns:\n  * " + '\n  * '.join(df.columns))
display(df.head(4).collect())

Dynamic Measurement Columns:
  * measurement_id
  * department
  * HR
  * temp
  * lab_name
  * lab_value
  * event_id
  * HR_is_inlier
  * temp_is_inlier
  * lab_name_is_inlier


measurement_id,department,HR,…,HR_is_inlier,temp_is_inlier,lab_name_is_inlier
u32,cat,f64,…,bool,bool,bool
0,"""CARDIAC""",null,…,null,null,null
1,"""PULMONARY""",null,…,null,null,null
2,"""PULMONARY""",null,…,null,null,null
3,"""PULMONARY""",null,…,null,null,null


The dynamic measurements dataframe `dynamic_measurements_df` has an ID column, an `event_id` linking column, and then all our measurements, recorded with missingness.

##### Fit Measurement Properties
In all of these dataframes, we can see the outputs from our learned vocabularies, outlier detector models, and normalizer models. How can we determine what fit parameters were used to make those distinctions? These data are stored in the `inferred_measurement_metadata` objects. The overall container is stored in `inferred_measurement_configs.json`, which contains an object linking measurement names to overall configs:

In [15]:
!cat processed/sample/inferred_measurement_configs.json | python -m json.tool

{
    "eye_color": {
        "name": "eye_color",
        "temporality": "static",
        "modality": "single_label_classification",
        "observation_frequency": 1.0,
        "functor": null,
        "vocabulary": {
            "vocabulary": [
                "UNK",
                "BROWN",
                "BLUE",
                "HAZEL",
                "GREEN"
            ],
            "obs_frequencies": [
                0.0,
                0.5125,
                0.2125,
                0.175,
                0.1
            ]
        },
        "values_column": null,
        "_measurement_metadata": null
    },
    "department": {
        "name": "department",
        "temporality": "dynamic",
        "modality": "multi_label_classification",
        "observation_frequency": 0.0007383747656851046,
        "functor": null,
        "vocabulary": {
            "vocabulary": [
                "UNK",
                "CARDIAC",
                "PULMONARY",
                "ORTHOP

We can see that these objects contain the full vocabularies learned, as well as (for numerical measurements) internal links to further measurement metadata `csv` files. These `csv` files contain more detailed statistics for numerical data, such as outlier detector and normalizer models. Let's inspect two of these, one for the `multivariate_regression` measurement `lab_name` and one for the `univariate_regression` measurement `age`:

In [16]:
!csvlook -d "," processed/sample/inferred_measurement_metadata/lab_name.csv

| lab_name   | value_type          | outlier_model                                                               | normalizer                                                |
| ---------- | ------------------- | --------------------------------------------------------------------------- | --------------------------------------------------------- |
| creatinine | float               | {'thresh_large_': 15940.942090515884, 'thresh_small_': -15715.577631719556} | {'mean_': 1.3238022046456384, 'std_': 1.0517759234408144} |
| SOFA       | categorical_integer | {'thresh_large_': None, 'thresh_small_': None}                              | {'mean_': None, 'std_': None}                             |
| potassium  | float               | {'thresh_large_': 12622.159050165134, 'thresh_small_': -12472.58665045867}  | {'mean_': 4.810794749300743, 'std_': 1.6868907048792603}  |
| GCS        | categorical_integer | {'thresh_large_': None, 'thresh_small_': None}                              | {'mean_': 

We can see that the `lab_name.csv` file contains a dataframe mapping the `lab_name` (the categorical component of this multivariate regression task) to the inferred `value_type` (whether the value is a `float`, `integer`, `categorical_float`, or `categorical_integer`), `outlier_model` parameters, and `normalizer` parameters. From this, we can see that the `GCS` lab test has been interpreted as a `categorical_integer`, and from the `vocabulary` in the prior JSON object, we can see that it takes on values ranging from 1 to 15. In contrast, we can see that the `SpO2` lab value is a float a mean of 50.9 (which, to be clear, is a _bad_ real-world SpO2), and has an inferred outlier threshold of approximately $\pm15000$. 

In [17]:
!csvlook -d "," --max-column-width=100 processed/sample/inferred_measurement_metadata/age.csv

/home/mmd/Programs/miniconda3/envs/RAMMS_cpu/lib/python3.10/site-packages/agate/utils.py:274: UnnamedColumnWarning: Column 0 has no name. Using "a".
| a             | age                                                                      |
| ------------- | ------------------------------------------------------------------------ |
| value_type    | float                                                                    |
| outlier_model | {'thresh_large_': 39.4032082341397, 'thresh_small_': 22.911637579463825} |
| normalizer    | {'mean_': 31.398075972111368, 'std_': 4.460737015690829}                 |


In contrast to the `multivariate_regression` measurement file, the `univariate` `age.csv` file contains a series representation mapping the three non-categorical-index columns of the prior file to their unique value for `age` alone. We can see that `age` is a floating point value, with a mean of $31.4\pm 4.5$ within the "inlier" range of $22.9 - 39.4$.

##### Inspecting the dataset object.
We can also look at the same content through the actual object oriented dataset interface, which contains all the above information and more, as loaded through some of the other files in this directory, such as `E.pkl` which contains other dataset attributes. Let's do this now.

In [20]:
import sys

sys.path.append("../")

In [21]:
# Imports
import os
from pathlib import Path

import polars as pl

from EventStream.data.dataset_polars import Dataset

pl.Config.set_tbl_cols(7);

In [22]:
dataset_dir = Path(os.getcwd()) / "processed/sample"

## Loading the Dataset

In [23]:
ESD = Dataset.load(dataset_dir)

In [24]:
ESD.subjects_df.head(3)

Loading subjects from /home/mmd/Projects/EventStreamGPT/sample_data/processed/sample/subjects_df.parquet...


subject_id,MRN,eye_color,dob
u8,cat,cat,datetime[μs]
0,"""310243""","""GREEN""",1981-07-28 00:00:00
1,"""384198""","""BROWN""",1985-04-15 00:00:00
2,"""520533""","""BROWN""",1979-04-15 00:00:00


In [25]:
ESD.events_df.head(3)

Loading events from /home/mmd/Projects/EventStreamGPT/sample_data/processed/sample/events_df.parquet...


event_id,subject_id,timestamp,event_type,age,age_is_inlier
u32,u8,datetime[μs],cat,f64,bool
0,0,2010-06-24 13:23:00,"""ADMISSION&VITA…",-0.558276,true
1,0,2010-06-24 14:23:00,"""VITAL&LAB""",-0.55825,true
2,0,2010-06-24 15:23:00,"""VITAL&LAB""",-0.558224,true


In [26]:
ESD.dynamic_measurements_df.filter(pl.col("event_id") == 0)

Loading dynamic_measurements from /home/mmd/Projects/EventStreamGPT/sample_data/processed/sample/dynamic_measurements_df.parquet...


measurement_id,department,HR,…,HR_is_inlier,temp_is_inlier,lab_name_is_inlier
u32,cat,f64,…,bool,bool,bool
189,"""PULMONARY""",null,…,null,null,null
1980,null,-0.865191,…,true,true,null
2112,null,-1.006687,…,true,true,null
23396,null,-0.840872,…,true,true,null
36963,null,-1.128285,…,true,true,null
83676,null,null,…,null,null,true
107279,null,null,…,null,null,true
315525,null,null,…,null,null,true
1076810,null,null,…,null,null,true


### Other Files
#### `.logs`

This sub-directory contains a hydra run log file for this dataset build run. As the pipeline currently doesn't
take advantage of the python logging module at all, the files it contains are empty.

##### `E.pkl`

This file contains the class object itself and a collection of its attributes. It, importantly, _does not_
contain the nested dataframes (`subjects_df`, `events_df`, `dynamic_measurements_df`), as these are stored in
the files mentioned above and loaded lazily by the object during use, and similarly does not store the learned
measurement metadata files also mentioned above, which are lazily loaded in the same way.

```{warning}
Currently, this lazy saving/loading uses absolute paths, which makes transferring datasets to new locations more challenging! Paths can be modified in the class and config object locally to fix this problem, but it is a challenge.
```